In [1]:
import sys
print(sys.path)

['/opt/homebrew/anaconda3/envs/python-notebook/lib/python39.zip', '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9', '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/lib-dynload', '', '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages', '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/setuptools/_vendor']


# GPU check

## Env Setup

In [2]:
!python3 -m pip install asitop -q

# Tensorflow

In [3]:
import tensorflow as tf
print("device", tf.config.list_physical_devices('GPU'))
gpu_available = tf.test.is_gpu_available()
is_cuda_gpu_available = tf.test.is_gpu_available(cuda_only=True)
is_cuda_gpu_min_3 = tf.test.is_gpu_available(True, (3,0))
print(f'gpu available: {gpu_available}')
print(f'cuda available: {is_cuda_gpu_available}')
print(f'is cuda min 3: {is_cuda_gpu_min_3}')

device [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
gpu available: True
cuda available: True
is cuda min 3: False
Metal device set to: 

2024-09-29 12:18:35.000327: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-29 12:18:35.000416: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-09-29 12:18:35.001697: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-29 12:18:35.001707: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-09-29 12:18:35.001757: I tensor

Apple M2

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



In [4]:
import tensorflow as tf

print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

print("TensorFlow version:", tf.__version__)

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.9.0


In [7]:
%%time
# 2min 33s, AMD EPYC 7B13 
# 3min 17s, Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz

import tensorflow_datasets as tfds

tf.config.list_physical_devices('GPU')
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

batch_size = 128
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.Conv2D(64, kernel_size=(3, 3),
                 activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#   tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)
model.fit(
    ds_train,
    epochs=12,
    validation_data=ds_test,
)

ERROR:absl:module 'tensorflow_datasets.core' has no attribute 'utils'
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/tensorflow_datasets/__init__.py", line 59, in <module>
    from tensorflow_datasets import audio
  File "/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/tensorflow_datasets/audio/__init__.py", line 19, in <module>
    from tensorflow_datasets.audio.commonvoice import CommonVoice
  File "/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/tensorflow_datasets/audio/commonvoice.py", line 27, in <module>
    import tensorflow_datasets.public_api as tfds
  File "/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/tensorflow_datasets/public_api.py", line 57, in <module>
    deprecated = core.utils.docs.deprecated(deprecated)
AttributeError: module 'tensorflow_datasets.core' has no attribute 'utils'


AttributeError: module 'tensorflow_datasets' has no attribute 'load'

# PyTorch

In [6]:

!python3 -m pip install -q fbgemm-gpu --index-url https://download.pytorch.org/whl/cu121
!python3 -m pip install -q torch torchvision torchaudio torchrec torchmetrics --index-url https://download.pytorch.org/whl/cu121
!python3 -m pip install torch==1.13.0+cu120 torchrec==0.1.0 -f https://download.pytorch.org/whl/cu120/torch_stable.htmldd

ERROR: Could not find a version that satisfies the requirement fbgemm-gpu (from versions: none)
ERROR: No matching distribution found for fbgemm-gpu
^C
ERROR: Operation cancelled by user
Looking in links: https://download.pytorch.org/whl/cu120/torch_stable.htmldd
ERROR: Could not find a version that satisfies the requirement torch==1.13.0+cu120 (from versions: 1.9.0, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch==1.13.0+cu120


In [3]:
import torch
import torch.utils.data
import os
import torchrec
import torch.distributed as dist

print(torch.cuda.is_available())
use_cuda = torch.cuda.is_available()
if use_cuda:
    print('__CUDNN VERSION:', torch.backends.cudnn.version())
    print('__Number CUDA Devices:', torch.cuda.device_count())
    print('__CUDA Device Name:',torch.cuda.get_device_name(0))
    print('__CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

# Note - you will need a V100 or A100 to run tutorial as as!
# If using an older GPU (such as colab free K80),
# you will need to compile fbgemm with the appripriate CUDA architecture nccl
# or run with "gloo" on CPUs 
dist.init_process_group(backend="gloo")

OSError: dlopen(/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/fbgemm_gpu/fbgemm_gpu_py.so, 0x0006): tried: '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/fbgemm_gpu/fbgemm_gpu_py.so' (not a mach-o file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/fbgemm_gpu/fbgemm_gpu_py.so' (no such file), '/opt/homebrew/anaconda3/envs/python-notebook/lib/python3.9/site-packages/fbgemm_gpu/fbgemm_gpu_py.so' (not a mach-o file)

In [9]:
%%time
# 500 ms in Quadro RTX 5000
# 600 ms in MPS m2
# 12900 ms Intel(R) Xeon(R) Gold 5218 CPU @ 2.30GHz
# 35800 ms Intel(R) Xeon(R) Silver 4214R CPU @ 2.40GHz
# 21700 ms AMD EPYC 7B13
# 13400 ms Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz

import torch

# Function to perform stress test on GPU
def stress_test_gpu(iterations=1000, tensor_size=(1024, 1024), device=torch.device('mps')):
    for i in range(iterations):
        if i % 100 == 0:
            print(f"Iteration: {i}")
        x = torch.rand(tensor_size, device=device)
        y = torch.rand(tensor_size, device=device)
        z = x * y
        del x, y, z

# Perform stress test
stress_test_gpu()

Iteration: 0
Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900
CPU times: user 164 ms, sys: 95.3 ms, total: 260 ms
Wall time: 430 ms


In [10]:
ebc = torchrec.EmbeddingBagCollection(
    device="meta",
    tables=[
        torchrec.EmbeddingBagConfig(
            name="product_table",
            embedding_dim=64,
            num_embeddings=4096,
            feature_names=["product"],
            pooling=torchrec.PoolingType.SUM,
        ),
        torchrec.EmbeddingBagConfig(
            name="user_table",
            embedding_dim=64,
            num_embeddings=4096,
            feature_names=["user"],
            pooling=torchrec.PoolingType.SUM,
        )
    ]
)

NameError: name 'torchrec' is not defined

## PyTorch M1 GPU Test
https://zhuanlan.zhihu.com/p/522534035

In [11]:
x = torch.ones((1024 * 12, 1024 * 12), dtype=torch.float32,device='mps')

RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, ort, mps, xla, lazy, vulkan, meta, hpu, privateuseone device type at start of device string: gpu

In [ ]:
%%time
import torch
import numpy as np

device = torch.device("mps")
print(device)
N = 3

A = torch.rand([N,N],dtype=torch.float32)*2 - 1
B = torch.rand([N,N],dtype=torch.float32)*5 - 2.5

C = A @ B
a = A.numpy().astype(np.float32)
b = B.numpy().astype(np.float32)
c = a @ b
C.numpy() - c
device = torch.device('mps')

aa = A.to(device)
bb = B.to(device)
cc = aa @ bb
CC = cc.cpu().numpy()

cc.cpu() - C


mps
CPU times: user 4.3 ms, sys: 7.35 ms, total: 11.7 ms
Wall time: 11.4 ms


tensor([[ 0.0000e+00, -5.9605e-08,  4.4703e-08],
        [-1.1921e-07,  0.0000e+00,  1.1921e-07],
        [-2.9802e-08,  0.0000e+00,  0.0000e+00]])

### PyTorch CPU to GPU
https://juejin.cn/post/7137891506777489416   
https://nni.readthedocs.io/en/stable/tutorials/pruning_bert_glue.html   

In [12]:
import torch
import torchvision

# device = torch.device("cuda:0")
device = torch.device("mps")

x = torch.randn(32, 32, device=device)
model = torchvision.models.resnet18().to(device)

print(x.device)
print(next(model.parameters()).device)


NotImplementedError: Could not run 'aten::empty.memory_format' with arguments from the 'MPS' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'aten::empty.memory_format' is only available for these backends: [Dense, FPGA, Conjugate, Negative, ZeroTensor, VmapMode, FuncTorchGradWrapper, Functionalize, MPS, IPU, UNKNOWN_TENSOR_TYPE_ID, QuantizedXPU, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseCPU, SparseCUDA, SparseHIP, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, SparseXPU, UNKNOWN_TENSOR_TYPE_ID, SparseVE, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, UNKNOWN_TENSOR_TYPE_ID, NestedTensorCUDA, UNKNOWN_TE

#### M1 GPU

In [13]:
torch.backends.mps.is_available()

False

In [ ]:
dev_mode=False

In [ ]:
from transformers import *
from torch.optim import *
from torch import *

In [ ]:
from pathlib import Path
# used to save GLUE data
data_dir = Path(f'./data')
data_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
from pathlib import Path
from typing import Callable, Dict

pretrained_model_name_or_path = 'bert-base-uncased'
task_name = 'mnli'
experiment_id = 'pruning_bert_mnli'

# heads_num and layers_num should align with pretrained_model_name_or_path
heads_num = 12
layers_num = 12

# used to save the experiment log
log_dir = Path(f'./pruning_log/{pretrained_model_name_or_path}/{task_name}/{experiment_id}')
log_dir.mkdir(parents=True, exist_ok=True)

# used to save the finetuned model and share between different experiemnts with same pretrained_model_name_or_path and task_name
model_dir = Path(f'./models/{pretrained_model_name_or_path}/{task_name}')
model_dir.mkdir(parents=True, exist_ok=True)

# used to save GLUE data
data_dir = Path(f'./data')
data_dir.mkdir(parents=True, exist_ok=True)

# set seed
from transformers import set_seed
set_seed(1024)

import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import BertTokenizerFast, DataCollatorWithPadding

task_to_keys = {
    'cola': ('sentence', None),
    'mnli': ('premise', 'hypothesis'),
    'mrpc': ('sentence1', 'sentence2'),
    'qnli': ('question', 'sentence'),
    'qqp': ('question1', 'question2'),
    'rte': ('sentence1', 'sentence2'),
    'sst2': ('sentence', None),
    'stsb': ('sentence1', 'sentence2'),
    'wnli': ('sentence1', 'sentence2'),
}

def prepare_dataloaders(cache_dir=data_dir, train_batch_size=32, eval_batch_size=32):
    tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path)
    sentence1_key, sentence2_key = task_to_keys[task_name]
    data_collator = DataCollatorWithPadding(tokenizer)

    # used to preprocess the raw data
    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=False, max_length=128, truncation=True)

        if 'label' in examples:
            # In all cases, rename the column to labels because the model will expect that.
            result['labels'] = examples['label']
        return result

    raw_datasets = load_dataset('glue', task_name, cache_dir=cache_dir)
    for key in list(raw_datasets.keys()):
        if 'test' in key:
            raw_datasets.pop(key)

    processed_datasets = raw_datasets.map(preprocess_function, batched=True,
                                          remove_columns=raw_datasets['train'].column_names)

    train_dataset = processed_datasets['train']
    if task_name == 'mnli':
        validation_datasets = {
            'validation_matched': processed_datasets['validation_matched'],
            'validation_mismatched': processed_datasets['validation_mismatched']
        }
    else:
        validation_datasets = {
            'validation': processed_datasets['validation']
        }

    train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
    validation_dataloaders = {
        val_name: DataLoader(val_dataset, collate_fn=data_collator, batch_size=eval_batch_size) \
            for val_name, val_dataset in validation_datasets.items()
    }

    return train_dataloader, validation_dataloaders


train_dataloader, validation_dataloaders = prepare_dataloaders()

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /Users/bicheng.wang/MyFiles/Projects/python/python-notebook/test/data/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

In [ ]:
import functools
import time

import torch.nn.functional as F
from datasets import load_metric
from transformers.modeling_outputs import SequenceClassifierOutput


def training(model: torch.nn.Module,
             optimizer: torch.optim.Optimizer,
             criterion: Callable[[torch.Tensor, torch.Tensor], torch.Tensor],
             lr_scheduler: torch.optim.lr_scheduler._LRScheduler = None,
             max_steps: int = None,
             max_epochs: int = None,
             train_dataloader: DataLoader = None,
             distillation: bool = False,
             teacher_model: torch.nn.Module = None,
             distil_func: Callable = None,
             log_path: str = Path(log_dir) / 'training.log',
             save_best_model: bool = False,
             save_path: str = None,
             evaluation_func: Callable = None,
             eval_per_steps: int = 1000,
             device=None):

    assert train_dataloader is not None

    model.train()
    if teacher_model is not None:
        teacher_model.eval()
    current_step = 0
    best_result = 0

    total_epochs = max_steps // len(train_dataloader) + 1 if max_steps else max_epochs if max_epochs else 3
    total_steps = max_steps if max_steps else total_epochs * len(train_dataloader)

    print(f'Training {total_epochs} epochs, {total_steps} steps...')

    for current_epoch in range(total_epochs):
        for batch in train_dataloader:
            if current_step >= total_steps:
                return
            batch.to(device)
            outputs = model(**batch)
            loss = outputs.loss

            if distillation:
                assert teacher_model is not None
                with torch.no_grad():
                    teacher_outputs = teacher_model(**batch)
                distil_loss = distil_func(outputs, teacher_outputs)
                loss = 0.1 * loss + 0.9 * distil_loss

            loss = criterion(loss, None)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # per step schedule
            if lr_scheduler:
                lr_scheduler.step()

            current_step += 1

            if current_step % eval_per_steps == 0 or current_step % len(train_dataloader) == 0:
                result = evaluation_func(model) if evaluation_func else None
                with (log_path).open('a+') as f:
                    msg = '[{}] Epoch {}, Step {}: {}\n'.format(time.asctime(time.localtime(time.time())), current_epoch, current_step, result)
                    f.write(msg)
                # if it's the best model, save it.
                if save_best_model and (result is None or best_result < result['default']):
                    assert save_path is not None
                    torch.save(model.state_dict(), save_path)
                    best_result = None if result is None else result['default']


def distil_loss_func(stu_outputs: SequenceClassifierOutput, tea_outputs: SequenceClassifierOutput, encoder_layer_idxs=[]):
    encoder_hidden_state_loss = []
    for i, idx in enumerate(encoder_layer_idxs[:-1]):
        encoder_hidden_state_loss.append(F.mse_loss(stu_outputs.hidden_states[i], tea_outputs.hidden_states[idx]))
    logits_loss = F.kl_div(F.log_softmax(stu_outputs.logits / 2, dim=-1), F.softmax(tea_outputs.logits / 2, dim=-1), reduction='batchmean') * (2 ** 2)

    distil_loss = 0
    for loss in encoder_hidden_state_loss:
        distil_loss += loss
    distil_loss += logits_loss
    return distil_loss


def evaluation(model: torch.nn.Module, validation_dataloaders: Dict[str, DataLoader] = None, device=None):
    assert validation_dataloaders is not None
    training = model.training
    model.eval()

    is_regression = task_name == 'stsb'
    metric = load_metric('glue', task_name)

    result = {}
    default_result = 0
    for val_name, validation_dataloader in validation_dataloaders.items():
        for batch in validation_dataloader:
            batch.to(device)
            outputs = model(**batch)
            predictions = outputs.logits.argmax(dim=-1) if not is_regression else outputs.logits.squeeze()
            metric.add_batch(
                predictions=predictions,
                references=batch['labels'],
            )
        result[val_name] = metric.compute()
        default_result += result[val_name].get('f1', result[val_name].get('accuracy', 0))
    result['default'] = default_result / len(result)

    model.train(training)
    return result


evaluation_func = functools.partial(evaluation, validation_dataloaders=validation_dataloaders, device=device)


def fake_criterion(loss, _):
    return loss

In [ ]:
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from transformers import BertForSequenceClassification


def create_pretrained_model():
    is_regression = task_name == 'stsb'
    num_labels = 1 if is_regression else (3 if task_name == 'mnli' else 2)
    model = BertForSequenceClassification.from_pretrained(pretrained_model_name_or_path, num_labels=num_labels)
    model.bert.config.output_hidden_states = True
    return model


def create_finetuned_model():
    finetuned_model = create_pretrained_model()
    finetuned_model_state_path = Path(model_dir) / 'finetuned_model_state.pth'

    if finetuned_model_state_path.exists():
        finetuned_model.load_state_dict(torch.load(finetuned_model_state_path, map_location='cpu'))
        finetuned_model.to(device)
    elif dev_mode:
        pass
    else:
        steps_per_epoch = len(train_dataloader)
        training_epochs = 3
        optimizer = Adam(finetuned_model.parameters(), lr=3e-5, eps=1e-8)

        def lr_lambda(current_step: int):
            return max(0.0, float(training_epochs * steps_per_epoch - current_step) / float(training_epochs * steps_per_epoch))

        lr_scheduler = LambdaLR(optimizer, lr_lambda)
        training(finetuned_model, optimizer, fake_criterion, lr_scheduler=lr_scheduler,
                 max_epochs=training_epochs, train_dataloader=train_dataloader, log_path=log_dir / 'finetuning_on_downstream.log',
                 save_best_model=True, save_path=finetuned_model_state_path, evaluation_func=evaluation_func, device=device)
    return finetuned_model


finetuned_model = create_finetuned_model()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

NameError: name 'dev_mode' is not defined

In [ ]:
steps_per_epoch = len(train_dataloader)

# Set training steps/epochs for pruning.

if not dev_mode:
    total_epochs = 4
    total_steps = total_epochs * steps_per_epoch
    warmup_steps = 1 * steps_per_epoch
    cooldown_steps = 1 * steps_per_epoch
else:
    total_epochs = 1
    total_steps = 3
    warmup_steps = 1
    cooldown_steps = 1

# Initialize evaluator used by MovementPruner.

import nni
from nni.algorithms.compression.v2.pytorch import TorchEvaluator

movement_training = functools.partial(training, train_dataloader=train_dataloader,
                                      log_path=log_dir / 'movement_pruning.log',
                                      evaluation_func=evaluation_func, device=device)
traced_optimizer = nni.trace(Adam)(finetuned_model.parameters(), lr=3e-5, eps=1e-8)

def lr_lambda(current_step: int):
    if current_step < warmup_steps:
        return float(current_step) / warmup_steps
    return max(0.0, float(total_steps - current_step) / float(total_steps - warmup_steps))

traced_scheduler = nni.trace(LambdaLR)(traced_optimizer, lr_lambda)
evaluator = TorchEvaluator(movement_training, traced_optimizer, fake_criterion, traced_scheduler)

# Apply block-soft-movement pruning on attention layers.
# Note that block sparse is introduced by `sparse_granularity='auto'`, and only support `bert`, `bart`, `t5` right now.

from nni.compression.pytorch.pruning import MovementPruner

config_list = [{
    'op_types': ['Linear'],
    'op_partial_names': ['bert.encoder.layer.{}.attention'.format(i) for i in range(layers_num)],
    'sparsity': 0.1
}]

pruner = MovementPruner(model=finetuned_model,
                        config_list=config_list,
                        evaluator=evaluator,
                        training_epochs=total_epochs,
                        training_steps=total_steps,
                        warm_up_step=warmup_steps,
                        cool_down_beginning_step=total_steps - cooldown_steps,
                        regular_scale=10,
                        movement_mode='soft',
                        sparse_granularity='auto')
_, attention_masks = pruner.compress()
pruner.show_pruned_weights()

torch.save(attention_masks, Path(log_dir) / 'attention_masks.pth')

In [ ]:
%%time
import transformers

model=transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

optimizer = pytorch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = transformers.get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


device = 'mps'
model.to(device)

model.train()
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        if i>10:
            break


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


AttributeError: 'tuple' object has no attribute 'loss'

#### M1 CPU

In [ ]:
%%time

model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

optimizer = AdamW(model.parameters(), lr=2e-5)

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

device = 'cpu'
model.to(device)

model.train()
for epoch in range(num_epochs):
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

        if i>10:
            break


NameError: name 'AdamW' is not defined

In [ ]:
%%time
import torch
import transformers

# Load BERT tokenizer and model
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
model = transformers.AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')

# Set up input data
input_text = ["This is a positive sentence.", "This is a negative sentence."]
labels = [1, 0]

# Tokenize input data
input_ids = []
attention_masks = []
for text in input_text:
    encoded_dict = tokenizer.encode_plus(
                        text,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert input data to tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set up optimizer and training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 5
batch_size = 2

# Train the model
for epoch in range(epochs):
    for i in range(0, len(input_ids), batch_size):
        input_batch = input_ids[i:i+batch_size]
        mask_batch = attention_masks[i:i+batch_size]
        label_batch = labels[i:i+batch_size]

        optimizer.zero_grad()

        outputs = model(input_batch, attention_mask=mask_batch, labels=label_batch)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

# Evaluate the model
with torch.no_grad():
    model.eval()
    input_ids = input_ids.to(model.device)
    attention_masks = attention_masks.to(model.device)
    labels = labels.to(model.device)

    outputs = model(input_ids, attention_mask=attention_masks, labels=labels)
    loss = outputs.loss
    logits = outputs.logits

    predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

print(predicted_labels)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

[1 0]
CPU times: user 4.61 s, sys: 1.48 s, total: 6.08 s
Wall time: 3.28 s


In [ ]:
%%time
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import load_dataset



# Load the IMDB dataset
dataset = load_dataset('imdb')

# Split the dataset into train and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(dataset['train']['text'], dataset['train']['label'], test_size=0.99, random_state=42)


Found cached dataset imdb (/Users/bicheng.wang/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 2.94 s, sys: 417 ms, total: 3.36 s
Wall time: 5.25 s


In [ ]:
# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize input data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert input data to tensors
train_inputs = {key: torch.tensor(val) for key, val in train_encodings.items()}
train_labels = torch.tensor(train_labels)
test_inputs = {key: torch.tensor(val) for key, val in test_encodings.items()}
test_labels = torch.tensor(test_labels)

# Set up optimizer and training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 1
batch_size = 64

device = torch.device("mps")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

#### Progress Bar

In [ ]:
# pip install tqdm
from tqdm import tqdm_notebook
# works on any iterable, including cursors.
# for iterables with len(), no need to specify 'total'.
from tqdm import tqdm
my_list = list(range(100))
with tqdm(total=len(my_list)) as pbar:
    for x in my_list:
        pbar.update(1)

100%|█████████████████████████████████████| 100/100 [00:00<00:00, 629775.38it/s]


In [ ]:
batch_size=1

In [ ]:
subsample = {'train': 1000, 'test': 100, 'batch_size': 10}
print("total:", len(dataset['']))
print(subsample)

total: 3
{'train': 1000, 'test': 100, 'batch_size': 10}


In [ ]:
%%time

print("start train")
# Train the model
model.train()

print("start epochs")
# with tqdm(total=epochs*range(0, len(train_labels[0:64]), batch_size)) as pbar:
for epoch in range(epochs):
    for i in range(0, len(train_labels[:subsample['train']]), subsample['batch_size']):
        inputs = {key: val[i:i+batch_size] for key, val in train_inputs.items()}
        labels = train_labels[i:i+batch_size]

        optimizer.zero_grad()

        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

#             pbar.update(1)

print("start evaluation")
# Evaluate the model
model.eval()
with torch.no_grad():

    inputs = {key: val[:subsample['test']].to(device) for key, val in test_inputs.items()}
    labels = test_labels[:subsample['test']].to(device)

    outputs = model(**inputs)
    logits = outputs.logits

predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
print(predicted_labels)

start train
start epochs
start evaluation
[0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
CPU times: user 9.76 s, sys: 4.89 s, total: 14.7 s
Wall time: 14.6 s


In [ ]:
# Load BERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Tokenize input data
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert input data to tensors
train_inputs = {key: torch.tensor(val) for key, val in train_encodings.items()}
train_labels = torch.tensor(train_labels)
test_inputs = {key: torch.tensor(val) for key, val in test_encodings.items()}
test_labels = torch.tensor(test_labels)

# Set up optimizer and training parameters
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
epochs = 1
batch_size = 64

device = torch.device("cpu")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
%%time

print("start train")
# Train the model
model.train()

print("start epochs")
# with tqdm(total=epochs*range(0, len(train_labels[0:64]), batch_size)) as pbar:
for epoch in range(epochs):
    for i in range(0, len(train_labels[:subsample['train']]), subsample['batch_size']):
        inputs = {key: val[i:i+batch_size] for key, val in train_inputs.items()}
        labels = train_labels[i:i+batch_size]

        optimizer.zero_grad()

        inputs = {key: val.to(device) for key, val in inputs.items()}
        labels = labels.to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

#             pbar.update(1)

print("start evaluation")
# Evaluate the model
model.eval()
with torch.no_grad():

    inputs = {key: val[:subsample['test']].to(device) for key, val in test_inputs.items()}
    labels = test_labels[:subsample['test']].to(device)

    outputs = model(**inputs)
    logits = outputs.logits

predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()
print(predicted_labels)

start train
start epochs
start evaluation
[1 1 0 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 1 1 1 0 1 0 1 0 0 1 1 0 0 1 0
 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1
 1 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 1 0]
CPU times: user 34min 32s, sys: 13min 50s, total: 48min 22s
Wall time: 16min 38s


In [1]:
torch.device("mps")

NameError: name 'torch' is not defined